В этом задании вы сможете немного расслабиться после реализации случайного леса и градиентного бустинга по мотивам материалов прошлой недели. Все, что нужно будет делать — запускать методы из sklearn. Вам предлагается выяснить, какое распределение лучше использовать в наивном байесовском классификаторе в зависимости от вида признаков.

Загрузите датасеты digits и breast_cancer из sklearn.datasets. Выведите несколько строчек из обучающих выборок и посмотрите на признаки. С помощью sklearn.cross_validation.cross_val_score c настройками по умолчанию и вызова метода mean() у возвращаемого этой функцией numpy.ndarray, сравните качество работы наивных байесовских классификаторов на этих двух датасетах. Для сравнения предлагается использовать BernoulliNB, MultinomialNB и GaussianNB. Насколько полученные результаты согласуются с рекомендациями из лекций?

Два датасета, конечно, еще не повод делать далеко идущие выводы, но при желании вы можете продолжить исследование на других выборках (например, из UCI репозитория).

Для сдачи задания, ответьте на приведенные ниже вопросы.

In [6]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib
from matplotlib import pyplot as plt
from sklearn import ensemble, cross_validation, learning_curve, metrics , tree, naive_bayes
from sklearn.tree import DecisionTreeRegressor
from sklearn.datasets import load_digits, load_breast_cancer
from sklearn.cross_validation import cross_val_score
matplotlib.style.use('ggplot')
%matplotlib inline

In [7]:
digits = load_digits()
breast_cancer = load_breast_cancer()

In [8]:
print(digits['DESCR'])

Optical Recognition of Handwritten Digits Data Set

Notes
-----
Data Set Characteristics:
    :Number of Instances: 5620
    :Number of Attributes: 64
    :Attribute Information: 8x8 image of integer pixels in the range 0..16.
    :Missing Attribute Values: None
    :Creator: E. Alpaydin (alpaydin '@' boun.edu.tr)
    :Date: July; 1998

This is a copy of the test set of the UCI ML hand-written digits datasets
http://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where
each class refers to a digit.

Preprocessing programs made available by NIST were used to extract
normalized bitmaps of handwritten digits from a preprinted form. From a
total of 43 people, 30 contributed to the training set and different 13
to the test set. 32x32 bitmaps are divided into nonoverlapping blocks of
4x4 and the number of on pixels are counted in each block. This generates
an input matrix of 8x8 where each element is a

In [9]:
print(breast_cancer['DESCR'])

Breast Cancer Wisconsin (Diagnostic) Database

Notes
-----
Data Set Characteristics:
    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry 
        - fractal dimension ("coastline approximation" - 1)
        
        The mean, standard error, and "worst" or largest (mean of the three
        largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 3 is Mean Radius, field
        13 is Radius SE, field 23 is Worst Radius.
 

Выведите несколько строчек из обучающих выборок и посмотрите на признаки. С помощью sklearn.cross_validation.cross_val_score c настройками по умолчанию и вызова метода mean() у возвращаемого этой функцией numpy.ndarray, сравните качество работы наивных байесовских классификаторов на этих двух датасетах. Для сравнения предлагается использовать BernoulliNB, MultinomialNB и GaussianNB. Насколько полученные результаты согласуются с рекомендациями из лекций?

Два датасета, конечно, еще не повод делать далеко идущие выводы, но при желании вы можете продолжить исследование на других выборках (например, из UCI репозитория).

Для сдачи задания, ответьте на приведенные ниже вопросы.

In [24]:
X_digits = digits['data']
y_digits = digits['target']
X_breast_cancer = breast_cancer['data']
y_breast_cancer = breast_cancer['target']

print(X_digits)
print(X_breast_cancer)

[[  0.   0.   5. ...,   0.   0.   0.]
 [  0.   0.   0. ...,  10.   0.   0.]
 [  0.   0.   0. ...,  16.   9.   0.]
 ..., 
 [  0.   0.   1. ...,   6.   0.   0.]
 [  0.   0.   2. ...,  12.   0.   0.]
 [  0.   0.  10. ...,  12.   1.   0.]]
[[  1.79900000e+01   1.03800000e+01   1.22800000e+02 ...,   2.65400000e-01
    4.60100000e-01   1.18900000e-01]
 [  2.05700000e+01   1.77700000e+01   1.32900000e+02 ...,   1.86000000e-01
    2.75000000e-01   8.90200000e-02]
 [  1.96900000e+01   2.12500000e+01   1.30000000e+02 ...,   2.43000000e-01
    3.61300000e-01   8.75800000e-02]
 ..., 
 [  1.66000000e+01   2.80800000e+01   1.08300000e+02 ...,   1.41800000e-01
    2.21800000e-01   7.82000000e-02]
 [  2.06000000e+01   2.93300000e+01   1.40100000e+02 ...,   2.65000000e-01
    4.08700000e-01   1.24000000e-01]
 [  7.76000000e+00   2.45400000e+01   4.79200000e+01 ...,   0.00000000e+00
    2.87100000e-01   7.03900000e-02]]


Вопрос 1

Каким получилось максимальное качество классификации на датасете breast_cancer?

In [25]:
%%time
#digits
#вечественные числа
scores_breast_cancer = []
#1
bern_brc_clf = naive_bayes.BernoulliNB()
bern_brc_clf.fit(X_breast_cancer, y_breast_cancer)
scores_breast_cancer.append([cross_val_score(bern_brc_clf, X_breast_cancer, y_breast_cancer, cv=10).mean(), 'BernoulliNB'])
#2
mult_brc_clf = naive_bayes.MultinomialNB()
mult_brc_clf.fit(X_breast_cancer, y_breast_cancer)
scores_breast_cancer.append([cross_val_score(mult_brc_clf, X_breast_cancer, y_breast_cancer, cv=10).mean(), 'MultinomialNB'])
#3
gaus_brc_clf = naive_bayes.GaussianNB()
gaus_brc_clf.fit(X_breast_cancer, y_breast_cancer)
scores_breast_cancer.append([cross_val_score(gaus_brc_clf, X_breast_cancer, y_breast_cancer, cv=10).mean(), 'GaussianNB'])
#answer
print(scores_breast_cancer)

[[0.62742740471869329, 'BernoulliNB'], [0.8929349235156856, 'MultinomialNB'], [0.93867967332123392, 'GaussianNB']]
CPU times: user 128 ms, sys: 7.61 ms, total: 136 ms
Wall time: 36.5 ms


Вопрос 2

Каким получилось максимальное качество классификации на датасете digits?

In [27]:
%%time
#digits
#целые неорицательные числа
scores_digits = []
#1
bern_dig_clf = naive_bayes.BernoulliNB()
bern_dig_clf.fit(X_digits, y_digits)
scores_digits.append([cross_val_score(bern_dig_clf, X_digits, y_digits, cv=10).mean(), 'BernoulliNB'])
#2
mult_dig_clf = naive_bayes.MultinomialNB()
mult_dig_clf.fit(X_digits, y_digits)
scores_digits.append([cross_val_score(mult_dig_clf, X_digits, y_digits, cv=10).mean(), 'MultinomialNB'])
#3
gaus_dig_clf = naive_bayes.GaussianNB()
gaus_dig_clf.fit(X_digits, y_digits)
scores_digits.append([cross_val_score(gaus_dig_clf, X_digits, y_digits, cv=10).mean(), 'GaussianNB'])
#answer
print(scores_digits )

[[0.83185574649501604, 'BernoulliNB'], [0.88193962163008377, 'MultinomialNB'], [0.81035375835678214, 'GaussianNB']]
CPU times: user 340 ms, sys: 12 ms, total: 352 ms
Wall time: 92.1 ms


Вопрос 3

Выберите верные утверждения и запишите их номера через пробел (в порядке возрастания номера):

-1) На вещественных признаках лучше всего сработал наивный байесовский классификатор с распределением Бернулли

-2) На вещественных признаках лучше всего сработал наивный байесовский классификатор с мультиномиальным распределением

+3) Мультиномиальное распределение лучше показало себя на выборке с целыми неотрицательными значениями признаков

+4) На вещественных признаках лучше всего сработало нормальное распределение